In [1]:
from mars_gym.data import utils

ModuleNotFoundError: No module named 'torch.autocast_mode'

In [ ]:
utils.datasets()

In [3]:
df, df_metadata = utils.load_dataset('processed_trivago_rio')

3.55kKB [00:02, 1.55kKB/s]                           
605KB [00:00, 842KB/s]                          


In [4]:
df.loc[0,'impressions']

"['109351', '150138', '4345728', '105014', '478121', '104677', '4342016', '4304686', '5066908', '5493194', '104917', '5659850', '104789', '1223390', '2788294', '104765', '1477057', '153550', '5660226', '3497814', '4155306', '960255', '5196406', '829671', '5497220']"

In [5]:
df_metadata = df_metadata.iloc[:,1:].copy()

In [6]:
df_metadata.head()

,item_id,1 star,2 star,3 star,4 star,5 star,accessible hotel,accessible parking,adults only,air conditioning,...,terrace (hotel),theme hotel,towels,very good rating,volleyball,washing machine,wheelchair accessible,wifi (public areas),wifi (rooms),list_metadata
0,3217002,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..."
1,3486086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5497220,0,0,0,1,0,1,0,0,1,...,0,0,0,1,0,0,1,1,1,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, ..."
3,6551710,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, ..."
4,6680288,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, ..."


In [7]:
df.columns

Index(['session_id', 'user_id', 'timestamp', 'action_type', 'item_id',
       'impressions', 'list_reference_item', 'pos_item_id', 'clicked'],
      dtype='object')

In [8]:
df_metadata.columns.tolist()

['item_id',
 '1 star',
 '2 star',
 '3 star',
 '4 star',
 '5 star',
 'accessible hotel',
 'accessible parking',
 'adults only',
 'air conditioning',
 'airport hotel',
 'airport shuttle',
 'balcony',
 'bathtub',
 'beach',
 'beach bar',
 'beauty salon',
 'bed & breakfast',
 'bike rental',
 'boat rental',
 'body treatments',
 'boutique hotel',
 'bowling',
 'bungalows',
 'business centre',
 'business hotel',
 'cable tv',
 'car park',
 'casa rural (es)',
 'central heating',
 'childcare',
 'club hotel',
 'computer with internet',
 'concierge',
 'conference rooms',
 'convenience store',
 'convention hotel',
 'cosmetic mirror',
 'cot',
 'country hotel',
 'deck chairs',
 'design hotel',
 'desk',
 'direct beach access',
 'diving',
 'doctor on-site',
 'eco-friendly hotel',
 'electric kettle',
 'excellent rating',
 'express check-in / check-out',
 'family friendly',
 'fan',
 'farmstay',
 'fitness',
 'flatscreen tv',
 'free wifi (combined)',
 'free wifi (public areas)',
 'free wifi (rooms)',
 'fridg

In [123]:
!mkdir test
!mkdir out

mkdir: cannot create directory ‘test’: File exists
mkdir: cannot create directory ‘out’: File exists


In [9]:
import torch

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [11]:
import luigi
# import pandas as pd
# import os
# DATASET_DIR = 'test/'
# OUTPUT_PATH = 'out/'

In [12]:
import pandas as pd
from samples.trivago_simple.data import DATASET_DIR, OUTPUT_PATH
import os
from mars_gym.data.utils import DownloadDataset

class PrepareInteractionData(luigi.Task):
    def requires(self):
        return DownloadDataset(dataset="processed_trivago_rio", output_path=OUTPUT_PATH)

    def output(self):
        return luigi.LocalTarget(os.path.join(DATASET_DIR, "dataset.csv",))

    def run(self):
        os.makedirs(DATASET_DIR, exist_ok=True)

        df = pd.read_csv(self.input()[0].path)

        # .... transform dataset

        df.to_csv(self.output().path)


class PrepareMetaData(luigi.Task):
    def requires(self):
        return DownloadDataset(dataset="processed_trivago_rio", output_path=OUTPUT_PATH)

    def output(self):
        return luigi.LocalTarget(os.path.join(DATASET_DIR, "metadata.csv",))

    def run(self):
        os.makedirs(DATASET_DIR, exist_ok=True)

        df = pd.read_csv(self.input()[1].path)

        # .... transform dataset

        df.to_csv(self.output().path, index="item_id")

In [13]:
from mars_gym.data.task import BasePrepareDataFrames

class PrepareTrivagoDataFrame(BasePrepareDataFrames):
    def requires(self):
        return (
            PrepareInteractionData(),
            PrepareMetaData(),
        )

    @property
    def timestamp_property(self) -> str:
        return "timestamp"

    @property
    def dataset_dir(self) -> str:
        return DATASET_DIR

    @property
    def read_data_frame_path(self) -> pd.DataFrame:
        return self.input()[0].path

    @property
    def metadata_data_frame_path(self):
        return self.input()[1].path

In [1]:
from samples.trivago_simple.data import PrepareTrivagoDataFrame
import luigi
job = PrepareTrivagoDataFrame(item_column='item_id')
luigi.build([job], local_scheduler=True)

C:\Users\Rodrigues Neto\anaconda3\envs\mars-gym\lib\site-packages\luigi\parameter.py:283: UserWarning: Parameter "column_stratification" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
DEBUG: Checking if PrepareTrivagoDataFrame(session_test_size=0.1, test_size=0.2, sample_size=-1, minimum_interactions=5, dataset_split_method=time, column_stratification=None, test_split_type=random, item_column=item_id, available_arms_column_name=available_arms, n_splits=10, split_index=0, val_size=0.2, sampling_strategy=none, balance_fields=[], sampling_proportions={}, use_sampling_in_validation=False, eq_filters={}, neq_filters={}, isin_filters={}, seed=42) is complete
INFO: Informed scheduler that task   PrepareTrivagoDataFrame_available_arms____None_c76cd6b0de   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: Ther

True

In [15]:
[o.path for o in job.output()]

['output\\processed_trivago_rio\\dataset\\train_0.20_test=random_42_none_PrepareTrivagoDataFrame_available_arms____None_c76cd6b0de.csv',
 'output\\processed_trivago_rio\\dataset\\val_0.20_test=random_42_PrepareTrivagoDataFrame_available_arms____None_c76cd6b0de.csv',
 'output\\processed_trivago_rio\\dataset\\test_0.20_test=random_42_PrepareTrivagoDataFrame_available_arms____None_c76cd6b0de.csv',
 'output\\processed_trivago_rio\\dataset\\metadata.csv']

In [2]:
from mars_gym.data.dataset import InteractionsDataset
from mars_gym.meta_config import *
from samples.trivago_rio import data

trivago_rio = ProjectConfig(
    base_dir=data.BASE_DIR,
    prepare_data_frames_task=data.PrepareTrivagoDataFrame,
    dataset_class=InteractionsDataset,
    user_column=Column("user_id", IOType.INDEXABLE),
    item_column=Column("item_id", IOType.INDEXABLE),
    other_input_columns=[
        Column("pos_item_id", IOType.NUMBER),
        Column("list_reference_item", IOType.INDEXABLE_ARRAY, same_index_as="item_id"),
    ],
    metadata_columns=[Column("list_metadata", IOType.INT_ARRAY),],
    output_column=Column("clicked", IOType.NUMBER),
    available_arms_column_name="impressions"
)

In [3]:
import luigi
from typing import Dict, Any
import torch
import torch.nn as nn
from mars_gym.meta_config import ProjectConfig
from mars_gym.model.abstract import RecommenderModule


class SimpleLinearModel(RecommenderModule):
    def __init__(
        self,
        project_config: ProjectConfig,
        index_mapping: Dict[str, Dict[Any, int]],
    ):
      """
      build model architecture
      """
      super().__init__(project_config, index_mapping)
      #...

    def forward(
        self,
        user_ids: torch.Tensor,
        item_ids: torch.Tensor,
        pos_item_id: torch.Tensor,
        list_reference_item: torch.Tensor,
        list_metadata: torch.Tensor,
    ):
      """
      build forward
      """
      pass

In [4]:
class SimpleLinearModel(RecommenderModule):
    def __init__(
        self,
        project_config: ProjectConfig,
        index_mapping: Dict[str, Dict[Any, int]],
        n_factors: int,
        metadata_size: int,
        window_hist_size: int,
    ):
        super().__init__(project_config, index_mapping)

        self.user_embeddings = nn.Embedding(self._n_users, n_factors)
        self.item_embeddings = nn.Embedding(self._n_items, n_factors)

        # user + item + flatten hist + position + metadata
        num_dense = 2 * n_factors + window_hist_size * n_factors + 1 + metadata_size

        self.dense = nn.Sequential(
            nn.Linear(num_dense, 500), nn.SELU(), nn.Linear(500, 1),
        )

    def flatten(self, input: torch.Tensor):
        return input.view(input.size(0), -1)

    def forward(
        self,
        user_ids: torch.Tensor,
        item_ids: torch.Tensor,
        pos_item_id: torch.Tensor,
        list_reference_item: torch.Tensor,
        list_metadata: torch.Tensor,
    ):
        user_emb = self.user_embeddings(user_ids)
        item_emb = self.item_embeddings(item_ids)
        history_items_emb = self.item_embeddings(list_reference_item)

        x = torch.cat(
            (
                user_emb,
                item_emb,
                self.flatten(history_items_emb),
                pos_item_id.float().unsqueeze(1),
                list_metadata.float(),
            ),
            dim=1,
        )

        x = self.dense(x)
        return torch.sigmoid(x)

In [5]:
from mars_gym.model.bandit import BanditPolicy
from typing import Dict, Any, List, Tuple, Union

class BasePolicy(BanditPolicy):
    def __init__(self, reward_model: nn.Module, seed: int = 42):
        """
        Initialize bandit information and params
        """
        super().__init__(reward_model)

    def _select_idx(
        self,
        arm_indices: List[int],
        arm_contexts: Tuple[np.ndarray, ...] = None,
        arm_scores: List[float] = None,
        pos: int = 0,
    ) -> Union[int, Tuple[int, float]]:
        """
        Choose the index of arm selected in turn
        """

        return action

In [6]:
from numpy.random import RandomState


class EGreedyPolicy(BanditPolicy):
    def __init__(self, reward_model: nn.Module, seed: int = 42):
        super().__init__(reward_model)
        self._rng = RandomState(seed)

    def _select_idx(
        self,
        arm_indices: List[int],
        arm_contexts: Tuple[np.ndarray, ...] = None,
        arm_scores: List[float] = None,
        pos: int = 0,
    ) -> Union[int, Tuple[int, float]]:

        n_arms = len(arm_indices)
        arm_probas = np.ones(n_arms) / n_arms

        if self._rng.choice([True, False], p=[self._epsilon, 1.0 - self._epsilon]):
            action = self._rng.choice(len(arm_indices), p=arm_probas)
        else:
            action = int(np.argmax(arm_scores))

        return action
      

In [ ]:
from mars_gym.simulation.interaction import InteractionTraining
job_train = InteractionTraining(
    project="samples.trivago_simple.config.trivago_rio",
    recommender_module_class="samples.trivago_simple.simulation.SimpleLinearModel",
    recommender_extra_params={
        "n_factors": 10,
        "metadata_size": 148,
        "window_hist_size": 5,
    },
    bandit_policy_class="samples.trivago_simple.simulation.EGreedyPolicy",
    bandit_policy_params={
        "epsilon": 0.1,
        "seed": 42
    },
    test_size=0.99,
    obs_batch_size=25,
    num_episodes=1,
)
# luigi.build([job_train], local_scheduler=True)

In [ ]:
job_train.run()

In [ ]:
job_train.run()

In [ ]:
job_train.env.action_space.sample()

In [ ]:
job_train.env.action_space.sample()

In [137]:
job_train.env.reset()
next_state,reward,done,info = job_train.env.step(job_train.env.action_space.sample())

In [138]:
next_state

{'user_id': 1768,
 'pos_item_id': 1,
 'list_reference_item': [0, 0, 0, 0, 0],
 'impressions': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0,

In [139]:
reward

0.0

In [140]:
df

,session_id,user_id,timestamp,action_type,item_id,impressions,list_reference_item,pos_item_id,clicked
0,05fe82b496fb9,M1Z13DD0P2KH,1541422443,clickout item,4304686,"['109351', '150138', '4345728', '105014', '478...","['', '', '', '', '']",7,1.0
1,05fe82b496fb9,M1Z13DD0P2KH,1541422474,clickout item,960255,"['1475717', '5196406', '104880', '109351', '68...","['4304686', '', '', '', '']",20,1.0
2,05fe82b496fb9,M1Z13DD0P2KH,1541423039,clickout item,2188598,"['104558', '326781', '104786', '1223390', '206...","['4304686', '960255', '', '', '']",9,1.0
3,05fe82b496fb9,M1Z13DD0P2KH,1541424631,clickout item,8459162,"['105014', '5659850', '478121', '109351', '956...","['4304686', '960255', '2188598', '', '']",23,1.0
4,05fe82b496fb9,M1Z13DD0P2KH,1541424685,interaction item info,8459162,NaN,"['4304686', '960255', '2188598', '8459162', '']",-1,0.0
...,...,...,...,...,...,...,...,...,...
12104,fd29c8ef7413b,SC2JJQ2JA608,1541106223,clickout item,5708658,"['4415314', '4342016', '150138', '4345728', '5...","['4415314', '4415314', '4415314', '5708658', '...",6,1.0
12105,fd318152e4023,S9XY1GVBWNHM,1541110970,clickout item,2924213,"['3148916', '5790750', '104706', '97505', '292...","['', '', '', '', '']",4,1.0
12106,fd318152e4023,S9XY1GVBWNHM,1541111026,interaction item info,2924213,NaN,"['2924213', '', '', '', '']",-1,0.0
12107,fd318152e4023,S9XY1GVBWNHM,1541111089,clickout item,926343,"['104811', '926343', '104706', '8256462', '132...","['2924213', '2924213', '', '', '']",1,1.0


In [141]:
df_metadata

,item_id,1 star,2 star,3 star,4 star,5 star,accessible hotel,accessible parking,adults only,air conditioning,...,terrace (hotel),theme hotel,towels,very good rating,volleyball,washing machine,wheelchair accessible,wifi (public areas),wifi (rooms),list_metadata
0,3217002,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..."
1,3486086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5497220,0,0,0,1,0,1,0,0,1,...,0,0,0,1,0,0,1,1,1,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, ..."
3,6551710,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, ..."
4,6680288,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,9682444,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
814,2833190,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..."
815,5150678,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
816,5506362,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [142]:
job_train.run()

DataFrame: env_data_frame,  (8186, 5)
DataFrame: interactions_data_frame,  (10898, 9)


/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:32: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (778, 148)
  "A Box observation space has an unconventional shape (neither an image, nor a 1D vector). "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WAR

TypeError: ignored

In [ ]:
 !mars-gym run interaction \
--project samples.trivago_simple.config.trivago_rio \
--recommender-module-class samples.trivago_simple.simulation.SimpleLinearModel \
--recommender-extra-params '{"n_factors": 10, "metadata_size": 148, "window_hist_size": 5}' \
--bandit-policy-class samples.trivago_simple.simulation.EGreedyPolicy \
--bandit-policy-params '{"epsilon": 0.1}' \
--obs-batch-size 100

In [ ]:
import sys

In [ ]:
!python --version

In [ ]:
sys.path.insert(1,'samples')

In [ ]:
import samples

In [ ]:
type(sys.path)

In [ ]:
df